In [ ]:
%load_ext autoreload
%autoreload 2

# Predictions x Buildings Footprints

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rxr
from shapely.geometry import box
from src.data.microsoft_buildings import load_microsoft_unosat_buildings
from src.constants import CRS_GLOBAL, NO_DATA_VALUE
from src.data.unosat import get_unosat_geometry
import geopandas as gpd
import pandas as pd
from matplotlib.colors import ListedColormap
from tqdm import tqdm


def extract_raster_value(point, raster):
    value = raster.sel(x=point.x, y=point.y, method="nearest").item()
    return value


def get_buildings_with_preds_proba(aoi, folder, geo=None):
    # Load Microsoft buildings footprints
    buildings = load_microsoft_unosat_buildings(aoi)

    # Default to entire AOI
    if geo is None:
        geo = get_unosat_geometry(aoi)

    start_dates = pd.date_range("2020-06-01", "2022-06-01", freq="MS").strftime("%Y-%m-%d").tolist()
    for i, start_date in tqdm(enumerate(start_dates)):
        # Read predictions
        preds = rxr.open_rasterio(folder / f"{start_date}.tif").squeeze()

        if i == 0:
            # Clip buildings to geo and compute centroids
            geo_crs = gpd.GeoDataFrame(None, geometry=[geo], crs=CRS_GLOBAL).to_crs(preds.rio.crs).iloc[0].geometry
            buildings_ = buildings.to_crs(preds.rio.crs).copy()
            buildings_ = buildings_[buildings_.intersects(geo_crs)].clip(geo_crs).copy()
            buildings_["centroid"] = buildings_.geometry.centroid

        # Extract prediction values for each building
        preds_clipped = preds.rio.clip([geo_crs])
        buildings_[start_date] = buildings_["centroid"].apply(lambda x: extract_raster_value(x, preds_clipped))

    buildings_ = buildings_.replace(NO_DATA_VALUE, 0)
    return buildings_

## Get buildings with predictions probabilities

In [ ]:
from src.constants import LOGS_PATH
aoi = "UKR6"
# folder = Path("./preds_sliding_window_3x3_UKR6/")
folder = LOGS_PATH / "sliding_window_train_random15" / 'predictions' / aoi
geo = box(31.365, 51.525, 31.375, 51.53)
buildings_with_preds = get_buildings_with_preds_proba(aoi, folder, geo=geo)

## Visualize predictions

In [ ]:
from src.constants import DAMAGE_SCHEME, DAMAGE_COLORS
import matplotlib.patches as mpatches
import contextily as ctx
import ipywidgets as widgets
from ipywidgets import interact


def plot_buildings_unosat(buildings, ax=None, legend=True, damage_key="damage_2m", labels_to_keep=None, **kwargs):
    if buildings.empty:
        return ax

    if ax is None:
        _, ax = plt.subplots()

    # Plot destroyed buildings with corresponding color
    buildings_ = buildings[~buildings[damage_key].isna()]

    if labels_to_keep is not None:
        buildings_ = buildings_[buildings_[damage_key].isin(labels_to_keep)]

    for key, group in buildings_.groupby(damage_key):
        group.plot(color=DAMAGE_COLORS[key], ax=ax, **kwargs)

    # Plot buildings outline
    buildings.plot(ax=ax, facecolor="none", edgecolor="k")

    if legend:
        # Create a patch (proxy artist) for each damage category
        patches = [
            mpatches.Patch(color=DAMAGE_COLORS[key], label=DAMAGE_SCHEME[key])
            for key in sorted(buildings_[damage_key].unique())
        ]

        # Add legend to the plot
        ax.legend(handles=patches, title="Damage Categories", fontsize="small")

    ax.set_title("Msft Buildings with UNOSAT Labels")
    ctx.add_basemap(ax, crs=buildings.crs, zoom=14, source=ctx.providers.Stadia.StamenTonerLite)

    return ax


import matplotlib.cm as cm
import matplotlib.colors as colors


def plot_preds_msft(buildings, start_date, threshold=0.5, ax=None):
    if ax is None:
        _, ax = plt.subplots()

    im = buildings[buildings[start_date] > threshold].plot(
        start_date,
        cmap="YlOrRd",
        ax=ax,
        vmin=0.5,
        vmax=1.0,
        legend=True,
        legend_kwds={"label": "Probability", "fraction": 0.046, "pad": 0.04},
    )
    buildings.plot(ax=ax, facecolor="none", edgecolor="k")
    ax.set_title(f"Msft Buildings with Predictions - {start_date}")
    ctx.add_basemap(ax, crs=buildings.crs, zoom=14, source=ctx.providers.Stadia.StamenTonerLite) #CartoDB.VoyagerNoLabels)

    # Create a fake ScalarMappable for the colorbar
    # cmap = cm.YlOrRd
    # norm = colors.Normalize(vmin=0.5, vmax=1.0)
    # sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    # sm.set_array([])  # You can set an array of values here if you want different color ranges
    return ax


def plot_buildings_labels_vs_preds(start_date, threshold):
    _, axs = plt.subplots(1, 2, figsize=(20, 14))
    plot_buildings_unosat(buildings_with_preds, ax=axs[0], labels_to_keep=[1, 2])
    plot_preds_msft(buildings_with_preds, start_date=start_date, threshold=threshold, ax=axs[1])
    plt.show()


def plot_buildings_labels_vs_preds_slider():
    threshold_slider = widgets.SelectionSlider(
        options=[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85], description="Threshold", continuous_update=True
    )
    start_dates = pd.date_range("2020-06-01", "2022-06-01", freq="MS").strftime("%Y-%m-%d").tolist()
    dates_slider = widgets.SelectionSlider(options=start_dates, description="Start Date", continuous_update=True)
    interact(plot_buildings_labels_vs_preds, start_date=dates_slider, threshold=threshold_slider)

In [ ]:
plot_buildings_labels_vs_preds_slider()

In [ ]:
def plot_buildings_detroyed_over_time(buildings):
    _, ax = plt.subplots(figsize=(12, 5))

    # Bin values every 0.05
    bin_ranges = np.arange(0.5, 1.05, 0.05)
    bin_labels = [f"{bin_ranges[i]:.2f}-{bin_ranges[i+1]:.2f}" for i in range(len(bin_ranges) - 1)]
    bin_counts = {label: [] for label in bin_labels}

    start_dates = pd.date_range("2020-06-01", "2022-06-01", freq="MS").strftime("%Y-%m-%d").tolist()
    for start_date in start_dates:
        bin_series = pd.cut(buildings[start_date], bins=bin_ranges, labels=bin_labels, include_lowest=True)
        bin_counts_col = bin_series.value_counts().reindex(bin_labels, fill_value=0)
        for label in bin_labels:
            bin_counts[label].append(bin_counts_col[label])

    # Get same colormap as for preds
    cmap = plt.cm.YlOrRd
    colors = cmap(np.linspace(0, 1, len(bin_labels)))
    bin_counts_df = pd.DataFrame(bin_counts, index=start_dates)
    bin_counts_df.plot.area(stacked=True, ax=ax, color=colors)
    ax.legend(title='Probs of destruction')
    ax.set_title(f"Evolution of buildings destruction probability over time for {aoi}")
    ax.set_xlabel("Start Date")
    ax.set_ylabel("Percentage of Buildings with given probabilities")
    plt.show()

In [ ]:
plot_buildings_detroyed_over_time(buildings_with_preds)